# Using Cohere Command R and Command R+ Models on Amazon Bedrock
---
#### Introduction
In this notebook we will dive into how to get started with Cohere's Command and Command R+ on Bedrock and using the Bedrock invoke_model API specifically

---

#### Amazon Bedrock
Amazon Bedrock is a fully managed service that provides access to a wide range of powerful foundational models(FMs) through an API.

---

#### Cohere Command R and Command R+ models
Cohere Command R is a family of highly scalable language models that balance high performance with strong accuracy. Command R family – include Command R and Command R+ models – are optimized for RAG based workflows such as conversational interaction and long context tasks, enabling companies to move beyond proof of concept and into production. These powerful models are designed to handle complex tasks with high performance and strong accuracy, making them suitable for real-world applications.

Command R boasts high precision on RAG and tool use tasks, low latency and high throughput, a long 128,000-token context length, and strong capabilities across 10 key languages: English, French, Spanish, Italian, German, Portuguese, Japanese, Korean, Arabic, and Chinese.

Command R+ is the newest model, optimized for extremely performant conversational interaction and long-context tasks. It is recommended for workflows that lean on complex RAG functionality and multi-step tool use (agents), while Cohere R is well-suited for simpler RAG and single-step tool use tasks, as well as applications where price is a major consideration. The model is optimized to perform well in the following languages: English, French, Spanish, Italian, German, Brazilian Portuguese, Japanese, Korean, Simplified Chinese, and Arabic.

---

#### Prerequisites to using Cohere Command R+ and Command models available on Bedrock
1. Ensure you have requested access to the models provided by Cohere in the Bedrock console by clicking "model access." Instructions can be found here: https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html
2. Make sure you have the permissions to access Bedrock and you have the correct IAM permissions from your administrator 
3. Run the following cell to install boto3 and necessary packages

## Getting Started

## Install Dependencies

Here, we will install all the required dependencies to run this notebook.

In [ ]:
%pip install boto3 IPython --quiet

#### Now lets import the required modules to run the notbook and set up the Bedrock client

In [ ]:
import boto3
import json

## Select Model

Below we will set up the use of using Command R + models and Command R for the rest of our notebook. By default we will select Command R + and we also create the bedrock_rt client to use throughout the notebook.

In [ ]:
DEFAULT_MODEL= "cohere.command-r-plus-v1:0"
COMMAND_R_PLUS = "cohere.command-r-plus-v1:0"
COMMAND_R = "cohere.command-r-v1:0"
model_id = DEFAULT_MODEL

#### Now lets import the required modules to run the notbook and set up the Bedrock client

In [ ]:
bedrock_rt = boto3.client(service_name="bedrock-runtime")

---

The Cohere Command models have the following inference parameters available today. The one that is required is "message" parameter where the others are able to be added in a call to Bedrock where needed:

```
{
    "message": string,
    "chat_history": [
        {
            "role":"USER or CHATBOT",
            "message": string
        }
    ],
    "documents": [
        {"title": string, "snippet": string},
    ],
    "search_queries_only" : boolean,
    "preamble" : string,
    "max_tokens": int,
    "temperature": float,
    "p": float,
    "k": float,
    "prompt_truncation" : string,
    "frequency_penalty" : float,
    "presence_penalty" : float,
    "seed" : int,
    "return_prompt" : boolean,
    "tools" : [
        {
            "name": string,
            "description": string,
            "parameter_definitions": {
                "parameter name": {
                    "description": string,
                    "type": string,
                    "required": boolean
                }
            }
        }
    ],
    "tool_results" : [
        {
            "call": {
                "name": string,
                "parameters": {
                "parameter name": string
                }
            },
        "outputs": [
                {
                "text": string
                }
            ]
        }
    ],
    "stop_sequences": [string],
    "raw_prompting" : boolean

}
```

As you may notice above, there are quite a few inference parameters available for you to use on Bedrock for Cohere Command R model family specifically. In this notebook we wille explore creating an example function for how to call the invoke_model API as well as how to use a few of the above parameters. Subsequent notebooks will dive into examples of using more parameters. 

In [ ]:
#a function to generate the text
def generate_text(prompt, model_id, temp):
    body = {
    'message': prompt,
    'temperature': temp,
    'preamble':""
    }
# Invoke the Bedrock model
    response = bedrock_rt.invoke_model(
        modelId= model_id,
        body=json.dumps(body)
    )
# Print the response
    response_body = json.loads(response.get('body').read())
    #response_body = response_body['text'].replace("\n", " ")
    # Print the response  
    return response_body['text']

---

## Context
Providing context when we call Cohere LLMs helps drive a more accurate response. Let's see a few examples of different outputs when we give additional context vs when we don't.

### Without Context

In [ ]:
user_input ="What is a sample itinerary for a trip to Europe"
prompt = user_input
response = generate_text(prompt, model_id, .1)
print(response)

### With Context

In [ ]:
context = """Think back to the last time you were traveling around Europe. 
The trip will lasts 5 days and the first flight leaves seattle and lands in Nice, France at 4 PM on Monday. The flight returns on 3 PM on Friday back to Seattle. \
On the trip you want to go to the top resteraunts in Nice and visit near by areas in Nice for day trips. \
The time of year will be September so you are looking for active activities, eating good food and seeing the best views """

user_input = "What is a sample itinerary for a trip in Europe"

prompt = f"""{context}
Given the information above, answer this question: {user_input}"""

response = generate_text(prompt, model_id, temp=0.1)
print(response)

As you can see, if you provide more context to the models, they are able to understand more on the formatting or the structured response. There are multiple ways to add even more context for example, to say "only provide 2 suggestions of activities per day in Europe" to tailor the response event further.

---

### Prompting Techniques
Cohere developed core guidance for advanced prompting techniques for their models: https://docs.cohere.com/docs/advanced-prompt-engineering-techniques#few-shot-prompting. The following examples will step through few shot prompting and chain of thought prompting examples for Cohere Command R and Command R+. 

#### Few-Shot Prompting
- Few-shot prompting involves providing the model with a small number of examples demonstrating the desired task before asking it to perform that task on new inputs.
- Few-shot prompting is useful when you want the model to learn and generalize a specific task or skill from a small number of examples, rather than relying solely on its pre-training.
- This technique is also useful for when the task requires understanding a particular format, structure, or pattern that can be effectively conveyed through examples.
You need the model's outputs to closely match the style, tone, or conventions demonstrated in the examples.

---

##### Delimiters
In the below example you will also see the use of the delimiter '##'. Providing a well-structured and unambiguous prompt can enhance the performance of a large language model (LLM). It is beneficial to place the instructions at the start of the prompt and delineate different sections, such as instructions, context, and resources, using descriptive headers. These headers can be made more prominent by prefixing them with '##'. You can see in the below example that '## Examples' was used to show the start of the examples.


Let's see the below example for how you can use few-shot prompting to create outreach emails based on customer sentiment.




In [ ]:
examples = """
## Examples
Customer Sentiment: The customer had a negative experience with the product. They found it difficult to use and felt that the instructions were unclear.
Marketing Outreach Email:
Subject: We're sorry for the inconvenience

Dear [Customer Name],

We're sorry to hear that you had a negative experience with our product. At [Company Name], we strive to provide our customers with high-quality products and exceptional service.

We understand that the instructions may have been unclear, and we apologize for any frustration this may have caused. We value your feedback and would like to learn more about your experience to improve our product and documentation.

As a token of our appreciation for your patience and understanding, we'd like to offer you a 20% discount on your next purchase. Please use the code FEEDBACK20 at checkout.

If you have any further questions or concerns, please don't hesitate to reach out to our customer support team. We're here to help and ensure that you have a positive experience with our products.

Thank you for your continued support.

Best regards,
[Your Name]
[Company Name]

Customer Sentiment: The customer had a mixed experience with the product. They found some features useful but felt that others were lacking or confusing.
Marketing Outreach Email:
Subject: We value your feedback

Dear [Customer Name],

Thank you for sharing your experience with our product. We're glad to hear that you found some features useful, but we apologize for any confusion or frustration caused by the lacking or confusing aspects.

At [Company Name], we're committed to continuously improving our products and services based on customer feedback. Your input is invaluable in helping us identify areas for improvement and better meet your needs.

As a token of our appreciation for your feedback, we'd like to offer you a 15% discount on your next purchase. Please use the code VALUED15 at checkout.

We'd also like to invite you to schedule a call with one of our product specialists to discuss your experience in more detail. This will help us better understand your specific needs and concerns, and ensure that future updates address them effectively.

Thank you for choosing [Company Name], and we look forward to continuing to serve you in the future.

Best regards,
[Your Name]
[Company Name]

Customer Sentiment: The customer was frustrated with the product and felt that it did not meet their expectations. They had issues with the setup process and found the user interface confusing.
Marketing Outreach Email:
Subject: We're here to help

Dear [Customer Name],

We're sorry to hear that our product did not meet your expectations. At [Company Name], we take customer satisfaction seriously, and we want to ensure that you have a positive experience with our products and services.

We understand that the setup process and user interface caused frustration, and we apologize for any inconvenience this may have caused. We value your feedback and would like to learn more about your specific issues to improve our product and documentation.

As a token of our appreciation for your patience and understanding, we'd like to offer you a 25% discount on your next purchase. Please use the code SUPPORT25 at checkout.

Additionally, we'd like to invite you to schedule a one-on-one session with one of our product experts. During this session, they can walk you through the setup process, address any concerns you may have, and provide personalized guidance to help you get the most out of our product.

At [Company Name], we're committed to providing exceptional customer service, and we want to ensure that you have a positive experience moving forward.

Thank you for choosing [Company Name], and we look forward to addressing your concerns and exceeding your expectations.

Best regards,
[Your Name]
[Company Name]

"""

user_input = "The customer was generally satisfied with the product but felt that the pricing was too high for the features offered."

prompt = f""" ## Below there are a few examples of the creationg of marketing outreach email with an email subject based on a customer's sentiment. Do not include anything additional. Please generate an email based on user input following
the below examples {examples} to answer this question: {user_input}"""

response = generate_text(prompt, model_id, temp=0.2)
print(response)

---

### Steps
To steer the model toward generating higher-quality responses, it can be helpful to add instructions for the model to generate intermediate steps before generating the final output. The information generated during these steps helps enrich the model’s context before it generates the final response.

#### Without Steps

In [ ]:
user_input = "technology"

prompt = f"""Generate a startup idea for this industry: {user_input}"""

response = generate_text(prompt, model_id, temp=0.5)
print(response)

#### Without Steps

In [ ]:
user_input = "technology"

prompt = f"""Generate a startup idea for this industry: {user_input}
First, describe the problem to be solved.
Next, describe the target audience of this startup idea.
Next, describe the startup idea and how it solves the problem for the target audience.
Next, provide a name for the given startup.

Use the following format:
Industry: <the given industry>
The Problem: <the given problem>
Audience: <the given target audience>
Startup Idea: <the given idea>
Startup Name: <the given name>"""

response = generate_text(prompt, model_id, temp=0.9)
print(response)

---

#### RAG Examples

 Command R targets the emerging “scalable” category of models that balance high efficiency with strong accuracy, enabling companies to move beyond proof of concept, and into production. Command R is a generative model optimized for long context tasks such as retrieval-augmented generation (RAG). Command R+ is ideal for advanced Retrieval Augmented Generation (RAG) and also provides citation to reduce hallucinations. You may be wondering, What is RAG? Retrieval-Augmented Generation (RAG) is the process of optimizing the output of a large language model, so it references an authoritative knowledge base outside of its training data sources before generating a response. Large Language Models (LLMs) are trained on vast volumes of data and use billions of parameters to generate original output for tasks like answering questions, translating languages, and completing sentences. RAG extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, all without the need to retrain the model. It is a cost-effective approach to improving LLM output so it remains relevant, accurate, and useful in various contexts. Let's see an example below of this using the Bedrock API when we create a new function for helping us decide which AWS service to use:

In [ ]:
#a function to generate the text
def generate_RAG(prompt, model_id, temp):
    body = {
    'message': prompt,
    'temperature': temp,
    'preamble':"",
    'documents': [
    {
       "id": "s3_page",
       "title": "S3 product page",
       "snippet": "Amazon S3 is an object storage service offering industry-leading scalability, data availability, security, and performance",
       "url": "https://aws.amazon.com/s3/faqs/?nc=sn&loc=7",
    },
    {
        "id": "ec2_page",
        "title": "EC2 product page",
        "snippet": "Amazon EC2 is a web service that provides resizable compute capacity in the cloud. It is designed to make web-scale computing easier for developers.",
        "url": "https://aws.amazon.com/ec2/faqs/",
    }, 
    {
        "id": "dynamodb_page",
        "title": "DynamoDB product page",
        "snippet": "Amazon DynamoDB is a fully managed NoSQL database service that makes it easy to build and scale a real-time application.",
        "url": "https://aws.amazon.com/dynamodb/faqs/?refid=9eb02e4d-80e0-4f27-a621-b90b3c870bf3",
    }
    ]
    }
# Invoke the Bedrock model
    response = bedrock_rt.invoke_model(
        modelId= model_id,
        body=json.dumps(body)
    )
# Print the response
    response_body = json.loads(response.get('body').read())
    #response_body = response_body['text'].replace("\n", " ")
    # Print the response  
    responses = []
    responses = [response_body['text'], response_body['documents']]
    return responses

In [ ]:
prompt = "What service should I use to store PDFs?"

response = generate_RAG(prompt, model_id, 0.3)

#print the response and the citations that are returned from Bedrock API
print(response[0], "\n\n", "Citations", response[1], sep="")

In the above example, we only gave a high level description of S3 in the snippet but when we use the "URL" key, we are able to give more information. Then, the command R+ model takes care of the RAG approach for us since the model has been trained specifically for these types of use cases. In the Bedrock response, we can also return "response_body['documents]" which gives a high level citation for what the model referenced when giving the response.

---

##### Conclusion

In this notebook, we discussed examples of different prompting techniques when using the Cohere Command R and R+ models. Advanced prompting techniques like tool use and conversation history can all be explored to develop even stronger prompts per use case. We explored different base level techniques for Cohere's models that are similar to other general prompting techniques for other LLMs and models from other 3P model providers available on Bedrock today. 

Finally, we discussed using RAG based approach with Bedrock APIs. It’s worth noting that the prompting techniques discussed, including RAG, does not guarantee accuracy. We explored how providing context to the model is important to inform the model on how to reply but if the documents, data or even prompt information is out of date, the LLM will respond inaccurately. The processes discussed above help provide low lift ways to reduce risk and even with the feature of providing citations helps with the explainability of the model's response.